In [5]:
%matplotlib inline
import sys
import yfinance as yf
from datetime import datetime
from datetime import timedelta
import pandas as pd
import pandas_datareader as pdr
import datetime as dt
import matplotlib.pyplot as plt
import matplotlib.ticker as mtick
import pickle
import ipywidgets as widgets

In [2]:
fd_sora = pd.read_csv('fd_sora.csv',  parse_dates=[0])
fd_sora = fd_sora.dropna()
fd_sora = fd_sora.set_index('Date')
fd_sora.tail()

,Day,1 mo,2 mo,3 mo,6 mo,12 mo,Cpd SORA 3mo
Date,,,,,,,
14-Apr,Fri,0.0464,0.0471,0.0459,0.0446,0.0420,0.035928
17-Apr,Mon,0.0464,0.0471,0.0458,0.0444,0.0432,0.035898
18-Apr,Tue,0.0467,0.0474,0.0463,0.0452,0.0436,0.035851
19-Apr,Wed,0.0471,0.0478,0.0467,0.0459,0.0438,0.035816
20-Apr,Thu,0.0472,0.0479,0.0468,0.0460,0.0442,0.035802


In [3]:
fd_sora = fd_sora.drop(columns = ['Day',  'Cpd SORA 3mo']) #'Change', '% Change',
fd_sora.mul(100).tail()

,1 mo,2 mo,3 mo,6 mo,12 mo
Date,,,,,
14-Apr,4.64,4.71,4.59,4.46,4.20
17-Apr,4.64,4.71,4.58,4.44,4.32
18-Apr,4.67,4.74,4.63,4.52,4.36
19-Apr,4.71,4.78,4.67,4.59,4.38
20-Apr,4.72,4.79,4.68,4.60,4.42


In [4]:
fd_sora.info()

<class 'pandas.core.frame.DataFrame'>
Index: 39 entries, 27-Feb to 20-Apr
Data columns (total 5 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   1 mo    39 non-null     float64
 1   2 mo    39 non-null     float64
 2   3 mo    39 non-null     float64
 3   6 mo    39 non-null     float64
 4   12 mo   39 non-null     float64
dtypes: float64(5)
memory usage: 1.8+ KB


In [6]:
fd_sora.columns

Index(['1 mo', '2 mo', '3 mo', '6 mo', '12 mo'], dtype='object')

In [7]:
# Assume that fd_sora is a pandas DataFrame with dates as index and yields across maturities in different columns
# Assume that maturities is a list of column names for the yield data
maturities = ['1 mo', '2 mo', '3 mo', '6 mo', '12 mo']

# Define a function to update the plot based on the selected date
def update_plot(date):
    # Select the yield data for the selected date
    yield_data = fd_sora.loc[date, maturities]

    # Plot the yield curve
    fig, ax = plt.subplots(figsize=(10, 6))
    plt.plot(yield_data.index, yield_data.values*100)
    plt.title('DBS Fixed Deposit Rates on ' + str(date))
    plt.xlabel('Maturity')
    plt.ylabel('Yield (%)')


    # Add labels to each data point
    for x, y in zip(yield_data.index, yield_data.values*100):
        plt.text(x, y+0.01, str(round(y, 2)), ha='center', va='bottom', fontsize=8)

    plt.xticks(rotation=45)
    plt.show()

    
# Create a dropdown widget for selecting the date
date_widget = widgets.Dropdown(options=fd_sora.index, description='Date:')

# Use the interactive function to update the plot when the dropdown value changes
widgets.interactive(update_plot, date=date_widget)


interactive(children=(Dropdown(description='Date:', options=('27-Feb', '28-Feb', '1-Mar', '2-Mar', '3-Mar', '6…

In [8]:
import quandl as ql

# Set your API key
ql.ApiConfig.api_key = "<INSERT API KEY HERE>"

start_date = '2023-02-28'
end_date = '2023-04-13' #or can use dt.datetime.today()  for up to today

yield_ = ql.get("USTREASURY/YIELD", start_date=start_date, end_date=end_date, 
                api_key= "<INSERT API KEY HERE>")
yield_.to_pickle("yield_")
yield_ = yield_.dropna().div(100)
yield_.head()

,1 MO,2 MO,3 MO,6 MO,1 YR,2 YR,3 YR,5 YR,7 YR,10 YR,20 YR,30 YR
Date,,,,,,,,,,,,
2023-02-28,0.0465,0.0481,0.0488,0.0517,0.0502,0.0481,0.0451,0.0418,0.0407,0.0392,0.0410,0.0393
2023-03-01,0.0467,0.0482,0.0490,0.0520,0.0506,0.0489,0.0461,0.0427,0.0417,0.0401,0.0417,0.0397
2023-03-02,0.0475,0.0480,0.0491,0.0518,0.0504,0.0489,0.0463,0.0432,0.0424,0.0408,0.0424,0.0403
2023-03-03,0.0475,0.0479,0.0491,0.0518,0.0503,0.0486,0.0460,0.0426,0.0415,0.0397,0.0412,0.0390
2023-03-06,0.0475,0.0479,0.0493,0.0522,0.0505,0.0489,0.0461,0.0427,0.0416,0.0398,0.0414,0.0392


In [9]:
yield_.columns

Index(['1 MO', '2 MO', '3 MO', '6 MO', '1 YR', '2 YR', '3 YR', '5 YR', '7 YR',
       '10 YR', '20 YR', '30 YR'],
      dtype='object')

In [10]:
yield_.index = yield_.index.strftime('%e-%b')
yield_.index

Index(['28-Feb', ' 1-Mar', ' 2-Mar', ' 3-Mar', ' 6-Mar', ' 7-Mar', ' 8-Mar',
       ' 9-Mar', '10-Mar', '13-Mar', '14-Mar', '15-Mar', '16-Mar', '17-Mar',
       '20-Mar', '21-Mar', '22-Mar', '23-Mar', '24-Mar', '27-Mar', '28-Mar',
       '29-Mar', '30-Mar', '31-Mar', ' 3-Apr', ' 4-Apr', ' 5-Apr', ' 6-Apr',
       ' 7-Apr', '10-Apr', '11-Apr', '12-Apr', '13-Apr'],
      dtype='object', name='Date')

In [11]:
yield_.head()

,1 MO,2 MO,3 MO,6 MO,1 YR,2 YR,3 YR,5 YR,7 YR,10 YR,20 YR,30 YR
Date,,,,,,,,,,,,
28-Feb,0.0465,0.0481,0.0488,0.0517,0.0502,0.0481,0.0451,0.0418,0.0407,0.0392,0.0410,0.0393
1-Mar,0.0467,0.0482,0.0490,0.0520,0.0506,0.0489,0.0461,0.0427,0.0417,0.0401,0.0417,0.0397
2-Mar,0.0475,0.0480,0.0491,0.0518,0.0504,0.0489,0.0463,0.0432,0.0424,0.0408,0.0424,0.0403
3-Mar,0.0475,0.0479,0.0491,0.0518,0.0503,0.0486,0.0460,0.0426,0.0415,0.0397,0.0412,0.0390
6-Mar,0.0475,0.0479,0.0493,0.0522,0.0505,0.0489,0.0461,0.0427,0.0416,0.0398,0.0414,0.0392
